In [8]:
import pandas as pd
import numpy as np
import joblib

from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [2]:
folds = joblib.load('folds.pkl')
folds


[(array([     0,      1,      2, ..., 629997, 629998, 629999],
        shape=(504000,)),
  array([     9,     16,     19, ..., 629976, 629987, 629993],
        shape=(126000,))),
 (array([     0,      1,      4, ..., 629997, 629998, 629999],
        shape=(504000,)),
  array([     2,      3,      6, ..., 629977, 629982, 629989],
        shape=(126000,))),
 (array([     0,      2,      3, ..., 629995, 629996, 629999],
        shape=(504000,)),
  array([     1,      4,     12, ..., 629994, 629997, 629998],
        shape=(126000,))),
 (array([     1,      2,      3, ..., 629997, 629998, 629999],
        shape=(504000,)),
  array([     0,      5,     10, ..., 629988, 629990, 629995],
        shape=(126000,))),
 (array([     0,      1,      2, ..., 629995, 629997, 629998],
        shape=(504000,)),
  array([     7,      8,     11, ..., 629992, 629996, 629999],
        shape=(126000,)))]

In [16]:
df = pd.read_csv('df_att.csv')
X = df.drop('Heart Disease', axis=1)
y = df['Heart Disease']
y = y.map({'Presence': 1, 'Absence': 0})

cat_features = list(X.select_dtypes(exclude='number').columns)

params = {'iterations': 1233, 'depth': 5, 'learning_rate': 0.05563228358640898, 'l2_leaf_reg': 1.2388924496891942, 'border_count': 157}





oof_preds = np.zeros(len(X))
print(oof_preds)

for fold, (X_train_idx, X_val_idx) in enumerate(folds):
    
    model = CatBoostClassifier(**params)
    model = model.fit(X.iloc[X_train_idx], y.iloc[X_train_idx], cat_features=cat_features, verbose=300)
    oof_preds[X_val_idx] = model.predict_proba(X.iloc[X_val_idx])[:,1]
    
    print(f'O modelo foi treinado no fold {fold}')
print(oof_preds)

[0. 0. 0. ... 0. 0. 0.]
0:	learn: 0.6383054	total: 332ms	remaining: 6m 48s
300:	learn: 0.2689247	total: 56s	remaining: 2m 53s
600:	learn: 0.2667146	total: 1m 54s	remaining: 2m
900:	learn: 0.2656132	total: 2m 56s	remaining: 1m 4s
1200:	learn: 0.2647190	total: 4m	remaining: 6.41s
1232:	learn: 0.2646443	total: 4m 7s	remaining: 0us
O modelo foi treinado no fold 0
0:	learn: 0.6380137	total: 214ms	remaining: 4m 23s
300:	learn: 0.2683069	total: 57.3s	remaining: 2m 57s
600:	learn: 0.2659622	total: 1m 56s	remaining: 2m 2s
900:	learn: 0.2647883	total: 2m 56s	remaining: 1m 5s
1200:	learn: 0.2639019	total: 3m 59s	remaining: 6.38s
1232:	learn: 0.2638079	total: 4m 6s	remaining: 0us
O modelo foi treinado no fold 1
0:	learn: 0.6381691	total: 216ms	remaining: 4m 25s
300:	learn: 0.2688114	total: 59.7s	remaining: 3m 4s
600:	learn: 0.2666162	total: 1m 59s	remaining: 2m 5s
900:	learn: 0.2654810	total: 3m 1s	remaining: 1m 6s
1200:	learn: 0.2645693	total: 4m 6s	remaining: 6.56s
1232:	learn: 0.2644736	total: 

In [13]:
oof_preds[X_val_idx]

array([0., 0., 0., ..., 0., 0., 0.], shape=(126000,))

In [18]:
np.save('oof_preds_catboost.npy', oof_preds)


In [17]:
roc_auc_score(y, oof_preds)

0.9554896500502426

In [11]:
test = pd.read_csv('test.csv')
test.head()

index = test.id


predict = model.predict_proba(df)[:,1]

predict

prediction_df = pd.DataFrame({'id': index, 'Heart Disease': predict})

prediction_df.to_csv('seconde_submission.csv', index=False)
